## Notebook 7: Mapping Soil Properties

**Why do some fields drain well and others stay muddy? The answer is underground.**

Walk across a farm after a rain and you'll notice some areas dry out quickly while others stay waterlogged for days. Drive across Missouri and the soil color changes from brown to red to gray. These differences aren't random - they reflect the underlying **soil properties** that took thousands of years to form.

Soil properties control:
- **What crops grow best** - corn loves well-drained loam; rice needs heavy clay
- **How water moves** - sandy soil drains fast; clay holds water
- **Nutrient availability** - pH affects which nutrients plants can access
- **Carbon storage** - organic-rich soils store more carbon

In this notebook, we'll:

- Map clay content, organic carbon, and pH across Missouri
- See how soil texture varies from the Ozarks to the river bottoms
- Compare soil properties to actual cropland patterns

This data comes from **SSURGO** (Soil Survey Geographic Database) and **OpenLandMap** - decades of soil scientists digging pits and analyzing samples, now available from space.

## Setup

Same initialization as previous notebooks.

In [12]:
%pip install -q geemap folium

Note: you may need to restart the kernel to use updated packages.


In [13]:
import ee
from google.cloud import storage

# Initialize Earth Engine
PROJECT = "eeps-geospatial"
BUCKET = "wustl-eeps-edc"
ee.Initialize(project=PROJECT)

import geemap.foliumap as geemap

print("Ready!")

Ready!


## What is SSURGO?

**SSURGO** (Soil Survey Geographic Database) is the most detailed soil mapping available for the United States. It's maintained by the USDA Natural Resources Conservation Service (NRCS).

**How was it made?**
- Soil scientists walked the land, dug pits, and classified soils
- Each soil type ("map unit") has measured properties: texture, drainage, chemistry
- Decades of work, now digitized and available in GIS format

**OpenLandMap** takes SSURGO and similar datasets worldwide to create global soil property maps at ~250m resolution. We'll use these gridded products today.

**Key soil properties:**

| Property | What it tells you | Agricultural importance |
|----------|-------------------|------------------------|
| **Clay content** | How fine the particles are | High clay = slow drainage, hard to till |
| **Sand content** | How coarse the particles are | High sand = fast drainage, low nutrients |
| **Organic carbon** | Decomposed plant material | More = better fertility, water holding |
| **pH** | Acidity/alkalinity | Affects nutrient availability |
| **Texture class** | Overall particle size mix | Loam is ideal for most crops |

## Define area of interest: Missouri

Missouri has diverse soils because of its varied geology:
- **Glaciated plains** in the north - deep, fertile, loess-derived soils
- **Ozark Highlands** in the south - thin, rocky, acidic soils
- **Alluvial bottomlands** along rivers - heavy clay, poorly drained
- **Mississippi River floodplain** (Bootheel) - rich but flood-prone

In [14]:
# Get Missouri boundary from TIGER
states = ee.FeatureCollection("TIGER/2018/States")
missouri = states.filter(ee.Filter.eq("NAME", "Missouri"))
mo_geometry = missouri.geometry()

# Center point for maps
center_lat = 38.5
center_lon = -92.5

# Key regions for comparison
regions = {
    "Northern Plains (Macon Co)": [-92.5, 39.8],
    "Ozarks (Shannon Co)": [-91.4, 37.1],
    "Bootheel (Dunklin Co)": [-90.1, 36.2],
    "Missouri River Bottom": [-92.8, 38.7],
}

print("Area of interest: Missouri")
print("We'll compare soil properties across different regions.")

Area of interest: Missouri
We'll compare soil properties across different regions.


## Load soil property datasets

We'll load three key properties from OpenLandMap:
1. **Clay content** (%) - affects drainage and workability
2. **Organic carbon** (g/kg) - indicates fertility and carbon storage
3. **Soil pH** - controls nutrient availability

Each dataset has multiple depth layers (0, 10, 30, 60, 100, 200 cm). We'll use the surface layer (0 cm) since it's most relevant for agriculture.

In [15]:
# Load OpenLandMap soil datasets - surface layer (b0 = 0cm depth)

# Clay content (weight fraction, %)
clay = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select("b0")

# Organic carbon (stored as g/kg * 10, so divide by 10 for actual values)
organic_carbon = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select("b0")

# pH in water (stored as pH * 10, so divide by 10 for actual pH)
ph = ee.Image("OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02").select("b0")

# Sand content for reference
sand = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select("b0")

# Clip to Missouri
clay_mo = clay.clip(mo_geometry)
oc_mo = organic_carbon.clip(mo_geometry)
ph_mo = ph.clip(mo_geometry)
sand_mo = sand.clip(mo_geometry)

print("Loaded soil property data:")
print("  - Clay content (%)")
print("  - Organic carbon (g/kg, scaled)")
print("  - Soil pH (scaled)")
print("  - Sand content (%)")

Loaded soil property data:
  - Clay content (%)
  - Organic carbon (g/kg, scaled)
  - Soil pH (scaled)
  - Sand content (%)


## Map 1: Clay content

Clay content is one of the most important soil properties for agriculture:

- **High clay (>35%)**: Heavy soils that hold water, hard to work when wet, crack when dry
- **Medium clay (20-35%)**: Good balance of drainage and water retention
- **Low clay (<20%)**: Sandy/silty soils that drain quickly but may need irrigation

In Missouri, expect to see:
- High clay in river bottoms and the Bootheel
- Lower clay in the Ozarks (thin, rocky soils)

In [16]:
# Clay visualization - brown palette
clay_palette = [
    '#ffffd4',  # Light cream - low clay (sandy)
    '#fed98e',  # Light tan
    '#fe9929',  # Orange
    '#d95f0e',  # Dark orange
    '#993404',  # Brown - high clay
]

clay_vis = {
    'min': 10,
    'max': 45,
    'palette': clay_palette
}

# Create map
m1 = geemap.Map(center=[center_lat, center_lon], zoom=7)

m1.addLayer(clay_mo, clay_vis, 'Clay Content (%)')

# Add state boundary
m1.addLayer(mo_geometry, {'color': 'black'}, 'Missouri', False)

m1.add_colorbar(clay_vis, label='Clay Content (%)', layer_name='Clay Content (%)')

m1

## Map 2: Soil organic carbon

Organic carbon is decomposed plant and animal material - the "dark" in dark soil. It's critical for:

- **Fertility**: Organic matter releases nutrients as it decomposes
- **Water holding**: Acts like a sponge to retain moisture
- **Soil structure**: Helps soil particles clump together
- **Climate**: Soils store more carbon than the atmosphere!

The data is stored as g/kg × 10, so values of 20 = 2.0 g/kg organic carbon.

In [17]:
# Organic carbon palette - darker = more carbon
oc_palette = [
    '#f7f7f7',  # Light gray - low carbon
    '#d9d9d9',  # Gray
    '#bdbdbd',  # Medium gray
    '#969696',  # Darker gray
    '#636363',  # Dark gray
    '#252525',  # Very dark - high carbon
]

oc_vis = {
    'min': 0,
    'max': 20,  # 2.0 g/kg actual
    'palette': oc_palette
}

# Create map
m2 = geemap.Map(center=[center_lat, center_lon], zoom=7)

m2.addLayer(oc_mo, oc_vis, 'Organic Carbon')

m2.add_colorbar(oc_vis, label='Organic Carbon (g/kg × 10)', layer_name='Organic Carbon')

m2

## Map 3: Soil pH

Soil pH determines which nutrients plants can access:

- **Acidic (pH < 6)**: Common in high-rainfall areas; limits phosphorus, calcium availability
- **Neutral (pH 6-7.5)**: Ideal for most crops; nutrients most available
- **Alkaline (pH > 7.5)**: Common in dry areas; can limit iron, zinc availability

Most Missouri soils are slightly acidic (pH 5-6.5) due to moderate rainfall leaching calcium from the soil.

In [18]:
# Convert pH to actual units (data is stored as pH × 10)
ph_actual = ph_mo.divide(10)

# pH palette - red (acidic) to green (alkaline)
ph_palette = [
    '#d73027',  # Red - very acidic
    '#fc8d59',  # Orange - acidic
    '#fee08b',  # Yellow - slightly acidic
    '#d9ef8b',  # Light green - neutral
    '#91cf60',  # Green - slightly alkaline
    '#1a9850',  # Dark green - alkaline
]

ph_vis = {
    'min': 4.5,   # pH 4.5
    'max': 8.0,   # pH 8.0
    'palette': ph_palette
}

# Create map
m3 = geemap.Map(center=[center_lat, center_lon], zoom=7)

m3.addLayer(ph_actual, ph_vis, 'Soil pH')

m3.add_colorbar(ph_vis, label='Soil pH', layer_name='Soil pH')

m3

## Compare soil properties to land use

Let's see how soil properties relate to actual farming patterns. We'll overlay clay content with the USDA Cropland Data Layer (CDL), which maps crop types from satellite imagery.

**Expected patterns:**
- High-clay river bottoms → soybeans, rice (in Bootheel)
- Well-drained loamy soils → corn, soybeans in rotation
- Thin Ozark soils → pasture and forest, not row crops

In [19]:
# Load USDA Cropland Data Layer
cdl = ee.ImageCollection("USDA/NASS/CDL")

# Get most recent year
cdl_2023 = cdl.filter(ee.Filter.eq('system:index', '2023')).first().select('cropland')
cdl_mo = cdl_2023.clip(mo_geometry)

# CDL class values for reference:
# 1 = Corn, 5 = Soybeans, 24 = Winter Wheat, 36 = Alfalfa
# 37 = Pasture, 63 = Forest, 111 = Water, 121-124 = Developed

print("Loaded 2023 Cropland Data Layer")
print("\nCommon crop codes:")
print("  1 = Corn")
print("  5 = Soybeans")
print("  24 = Winter Wheat")
print("  37 = Pasture/Hay")
print("  63 = Forest")

Loaded 2023 Cropland Data Layer

Common crop codes:
  1 = Corn
  5 = Soybeans
  24 = Winter Wheat
  37 = Pasture/Hay
  63 = Forest


In [20]:
# Side-by-side: Clay content vs Cropland
left_layer = geemap.ee_tile_layer(clay_mo, clay_vis, 'Clay Content')
right_layer = geemap.ee_tile_layer(cdl_mo, {}, 'Cropland 2023')

m4 = geemap.Map(center=[center_lat, center_lon], zoom=7)
m4.split_map(left_layer, right_layer)

print("Left: Clay content | Right: Cropland types")
print("Compare the patterns - where are the row crops vs forests?")
m4

Left: Clay content | Right: Cropland types
Compare the patterns - where are the row crops vs forests?


## Zoom in: County-level patterns

Let's look at Boone County (home of Columbia and Mizzou) to see field-scale soil patterns. This county has diverse soils from river bottoms to upland ridges.

In [21]:
# Get Boone County boundary
counties = ee.FeatureCollection("TIGER/2018/Counties")
boone = counties.filter(
    ee.Filter.And(
        ee.Filter.eq("STATEFP", "29"),  # Missouri FIPS
        ee.Filter.eq("NAME", "Boone")
    )
)
boone_geom = boone.geometry()

# Clip soil data to county
clay_boone = clay.clip(boone_geom)

# Create county map
m5 = geemap.Map(center=[38.95, -92.33], zoom=10)

m5.addLayer(clay_boone, clay_vis, 'Clay Content (%)')
m5.addLayer(boone_geom, {'color': 'black'}, 'Boone County', False)

m5.add_colorbar(clay_vis, label='Clay Content (%)', layer_name='Clay Content (%)')

print("Boone County - notice the variation from river bottoms to uplands")
m5

Boone County - notice the variation from river bottoms to uplands


## Compare soil properties by region

Let's extract actual values at our reference locations to see how different Missouri regions compare.

In [22]:
print("Soil Properties by Region:")
print("=" * 70)
print(f"{'Region':<30} {'Clay %':<12} {'OC (g/kg)':<12} {'pH':<10}")
print("-" * 70)

for name, coords in regions.items():
    point = ee.Geometry.Point(coords)
    
    # Sample each property
    clay_val = clay.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=250
    ).get('b0').getInfo()
    
    oc_val = organic_carbon.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=250
    ).get('b0').getInfo()
    
    ph_val = ph.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=250
    ).get('b0').getInfo()
    
    # Convert to actual values
    oc_actual = oc_val / 10 if oc_val else None
    ph_actual = ph_val / 10 if ph_val else None
    
    print(f"{name:<30} {clay_val:<12} {oc_actual:<12.1f} {ph_actual:<10.1f}")

print("=" * 70)
print("\nNotice how the Bootheel has high clay, while Ozarks are lower.")

Soil Properties by Region:
Region                         Clay %       OC (g/kg)    pH        
----------------------------------------------------------------------
Northern Plains (Macon Co)     23           0.7          5.2       
Ozarks (Shannon Co)            15           0.3          6.3       
Bootheel (Dunklin Co)          17           0.2          5.9       
Missouri River Bottom          22           0.2          6.1       

Notice how the Bootheel has high clay, while Ozarks are lower.


## What patterns do you notice?

Looking at the maps:

**Clay content:**
- Highest along the Missouri and Mississippi River floodplains
- The Bootheel (SE Missouri) has heavy clay - historically swampland
- Ozarks have lower clay - thin, rocky soils over limestone/dolomite

**Organic carbon:**
- Higher in northern Missouri - glaciated prairies accumulated organic matter
- Lower in the Ozarks - thin soils, forest rather than prairie
- Moderate in bottomlands - flooded too often to build up

**Soil pH:**
- More acidic in the Ozarks (high rainfall + forest)
- More neutral in the glaciated north
- River bottoms can be more alkaline (calcium from flooding)

**Connection to land use:**
- Row crops (corn, soybeans) dominate the deep, fertile soils of northern Missouri
- The Ozarks are mostly forest and pasture - soils too thin for cultivation
- The Bootheel grows rice, cotton, and soybeans suited to heavy clay

## Try it yourself

Ideas to explore:

1. **Your home county**: Change the county filter to examine soils where you grew up. How do the patterns match what you remember?

2. **Deeper soil layers**: The data has layers at 0, 10, 30, 60, 100, 200 cm depth. Try `select("b30")` or `select("b100")` to see how properties change with depth.

3. **Sand content**: Load the sand dataset and compare to clay. Where is soil sandy? How does that relate to crop patterns?
   ```python
   sand = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select("b0")
   ```

4. **Soil texture classes**: Load the texture classification:
   ```python
   texture = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02").select("b0")
   # Classes: 1=Clay, 2=Silty Clay, 3=Sandy Clay, 4=Clay Loam, etc.
   ```

5. **Compare states**: How do Missouri soils compare to Iowa (more glaciated) or Arkansas (more Ozark-like)?

6. **Historical perspective**: The Bootheel was drained in the early 1900s. Look at historical maps - how did converting swamp to farmland change the landscape?

7. **Satellite imagery overlay**: Load a Sentinel-2 image and compare visible soil color to the mapped properties. Do dark soils really have more organic carbon?